In [1]:
# Внимание!!! Важно, что бы файлы с данными и исполняемый файл находились в одной папке, 
# тогда пути к тестовым и тренировочным наборам будут содержать только имена файлов.
# 
# В пути к тренировочным и тестовым данным запрежается использовать абсалютную адресацию, 
# то есть адресацию, в которой присутствуют имена папок. Путь должен содержать только имя файла.
#
# Напоминание: под моделью машинного обучения понимаются все действия с исходными данными, 
# которые необходимо произвести, что бы сопоставить признаки целевому значению.

### Область работы 1 (библиотеки)

In [1]:
# Данный блок в области 1 НЕ выполняется преподавателем
# 
# данный блок предназначен только для подключения необходимых библиотек
# запрещается подключать библиотеки в других блоках
#
# установка дополнительных библиотек размещается прямо здесь (обязательно закоментированы)
#
# pip install
# pip install catboost
# pip install lightgbm
# pip install xgboost
# pip install shap

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectFromModel
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import lightgbm as lgb
import xgboost as xgb
import catboost as cb
import shap
import warnings
warnings.filterwarnings('ignore')

### Область работы 2 (поиск  модели .... )

In [ ]:
train = pd.read_csv('train_diab.csv')
test = pd.read_csv('test_diab.csv')

In [ ]:
columns_to_drop = ['ethnicity', 'education_level', 'employment_status']

# Удаляем из тренировочных данных
for col in columns_to_drop:
    if col in train.columns:
        train = train.drop(col, axis=1)

# Удаляем из тестовых данных
for col in columns_to_drop:
    if col in test.columns:
        test = test.drop(col, axis=1)

In [ ]:
# 3. Feature Engineering функция
def create_features(df):
    df = df.copy()
    
    # Медицинские соотношения
    df['bp_ratio'] = df['systolic_bp'] / (df['diastolic_bp'] + 1e-5)
    df['cholesterol_ratio'] = df['cholesterol_total'] / (df['hdl_cholesterol'] + 1e-5)
    df['ldl_hdl_ratio'] = df['ldl_cholesterol'] / (df['hdl_cholesterol'] + 1e-5)
    df['triglycerides_hdl_ratio'] = df['triglycerides'] / (df['hdl_cholesterol'] + 1e-5)
    
    # Категоризация на основе медицинских норм
    df['bmi_category'] = pd.cut(df['bmi'], 
                                 bins=[0, 18.5, 25, 30, 35, 40, 100], 
                                 labels=[0, 1, 2, 3, 4, 5])
    
    df['bp_category'] = np.where(
        (df['systolic_bp'] >= 140) | (df['diastolic_bp'] >= 90), 2,
        np.where((df['systolic_bp'] >= 130) | (df['diastolic_bp'] >= 85), 1, 0)
    )
    
    df['age_group'] = pd.cut(df['age'], 
                              bins=[0, 30, 40, 50, 60, 70, 100], 
                              labels=[0, 1, 2, 3, 4, 5])
    
    df['activity_level'] = pd.cut(df['physical_activity_minutes_per_week'],
                                   bins=[0, 30, 90, 150, 300, 1000],
                                   labels=[0, 1, 2, 3, 4])
    
    # Полиномиальные и взаимодействующие признаки
    df['bmi_age'] = df['bmi'] * df['age'] / 100
    df['bmi_cholesterol'] = df['bmi'] * df['cholesterol_total'] / 1000
    df['age_cholesterol'] = df['age'] * df['cholesterol_total'] / 1000
    df['waist_bmi'] = df['waist_to_hip_ratio'] * df['bmi']
    
    # Статистические признаки
    df['total_risk_score'] = (
        (df['bmi'] > 30).astype(int) +
        (df['age'] > 50).astype(int) +
        (df['systolic_bp'] > 140).astype(int) +
        (df['family_history_diabetes'] > 0).astype(int) +
        (df['physical_activity_minutes_per_week'] < 150).astype(int)
    )
    
    return df

# Применяем feature engineering
train = create_features(train)
test = create_features(test)

In [ ]:
# 4. Разделение признаков
TARGET = 'diagnosed_diabetes'
ID_COL = 'id'

# Удаляем ID и целевую переменную из признаков
X = train.drop([ID_COL, TARGET], axis=1)
y = train[TARGET]

In [ ]:
# Определяем типы признаков
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X.select_dtypes(include=['object', 'category']).columns.tolist()

# Категориальные признаки
categorical_features = [col for col in ['gender', 'income_level', 'smoking_status',
                                        'bmi_category', 'age_group', 'bp_category', 
                                        'activity_level']
                        if col in X.columns]

numeric_features = [col for col in numeric_features if col not in categorical_features]

In [ ]:
# 5. Создание pipeline с кросс-валидационным подходом
skf = StratifiedKFold(n_splits=5, shuffle=True)

# Предобработка для числовых признаков
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Предобработка для категориальных признаков
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Объединение трансформеров
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [ ]:
# 6. Feature selection с использованием SHAP
print("Начинаем feature selection с SHAP...")

# Сначала обучим модель на всех признаках для оценки важности
lgb_base = lgb.LGBMClassifier(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=5,
    n_jobs=-1,
    verbosity=-1
)

# Создаем временный pipeline для оценки важности признаков
temp_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', lgb_base)
])

# Обучаем на одном фолде для скорости
train_idx, val_idx = next(skf.split(X, y))
X_temp, y_temp = X.iloc[train_idx], y.iloc[train_idx]
temp_pipeline.fit(X_temp, y_temp)

# Получаем важность признаков через SHAP
explainer = shap.TreeExplainer(temp_pipeline.named_steps['classifier'])
transformed_features = temp_pipeline.named_steps['preprocessor'].transform(X.iloc[X_temp])
shap_values = explainer.shap_values(transformed_features)

In [ ]:
# Если бинарная классификация, shap_values будет списком [для класса 0, для класса 1]
if isinstance(shap_values, list):
    shap_values = np.abs(shap_values[1])  # берем значения для положительного класса
else:
    shap_values = np.abs(shap_values)

mean_shap_values = np.mean(shap_values, axis=0)
feature_names = numeric_features + list(temp_pipeline.named_steps['preprocessor']
                                       .named_transformers_['cat']
                                       .named_steps['onehot']
                                       .get_feature_names_out(categorical_features))

# Выбираем топ-N признаков по SHAP значениям
feature_importance_df = pd.DataFrame({
    'feature': feature_names,
    'shap_importance': mean_shap_values
}).sort_values('shap_importance', ascending=False)

top_features = feature_importance_df.head(50)['feature'].tolist()  # берем топ-50 признаков
print(f"Выбрано {len(top_features)} наиболее важных признаков")


In [ ]:
# 7. Создание финального pipeline с отобранными признаками
# Модели для стекинга
base_models = [
    ('lgbm', lgb.LGBMClassifier(
        n_estimators=1000,
        learning_rate=0.01,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.1,
        reg_lambda=0.1,
        n_jobs=-1,
        verbosity=-1
    )),
    ('xgb', xgb.XGBClassifier(
        n_estimators=1000,
        learning_rate=0.01,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.1,
        reg_lambda=0.1,
        n_jobs=-1,
        eval_metric='auc',
        use_label_encoder=False
    )),
    ('catboost', cb.CatBoostClassifier(
        iterations=1000,
        learning_rate=0.01,
        depth=6,
        l2_leaf_reg=3,
        verbose=False,
        thread_count=-1
    )),
    ('rf', RandomForestClassifier(
        n_estimators=500,
        max_depth=10,
        min_samples_split=5,
        min_samples_leaf=2,
        max_features='sqrt',
        n_jobs=-1
    ))
]

# Мета-модель
meta_model = LogisticRegression(
    C=0.1,
    max_iter=1000,
    n_jobs=-1
)

# Создаем стекинг классификатор
stacking_model = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_model,
    cv=3,
    n_jobs=-1,
    passthrough=True  # Используем исходные признаки + предсказания базовых моделей
)


In [ ]:
# 8. Калиброванный классификатор
calibrated_model = CalibratedClassifierCV(
    stacking_model,
    method='isotonic',
    cv=3
)


In [ ]:
# 9. Финальный pipeline
final_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('feature_selector', SelectFromModel(
        RandomForestClassifier(n_estimators=100),
        threshold='median'
    )),
    ('classifier', calibrated_model)
])


In [ ]:
# 10. Кросс-валидация с out-of-fold predictions
print("\nНачинаем кросс-валидацию...")
oof_predictions = np.zeros(len(X))
feature_importance_list = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
    print(f"\nФолд {fold}/5:")
    
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    
    # Обучаем pipeline
    final_pipeline.fit(X_train, y_train)
    
    # Предсказания на валидации
    val_preds = final_pipeline.predict_proba(X_val)[:, 1]
    oof_predictions[val_idx] = val_preds
    
    # Оценка на фолде
    fold_auc = roc_auc_score(y_val, val_preds)
    print(f"AUC на фолде {fold}: {fold_auc:.4f}")

In [ ]:
# 11. Финальная оценка
final_auc = roc_auc_score(y, oof_predictions)
print(f"\n{'='*50}")
print(f"Финальный OOF AUC: {final_auc:.4f}")
print(f"{'='*50}")

In [ ]:
# 12. Обучение на всех данных для финальной модели
print("\nОбучаем финальную модель на всех данных...")
final_pipeline.fit(X, y)

In [ ]:
# 13. Создание предсказаний для тестового набора
print("\nСоздание предсказаний для тестового набора...")

# Подготовка тестовых данных
X_test = test.drop([ID_COL], axis=1, errors='ignore')

# Проверяем, что все колонки из X есть в X_test
missing_cols = set(X.columns) - set(X_test.columns)
if missing_cols:
    print(f"Внимание: в тестовых данных отсутствуют колонки: {missing_cols}")
    # Добавляем недостающие колонки с нулевыми значениями
    for col in missing_cols:
        X_test[col] = 0
    # Упорядочиваем колонки в том же порядке, что и в X
    X_test = X_test[X.columns]

# Делаем предсказания
test_predictions = final_pipeline.predict_proba(X_test)[:, 1]

In [ ]:
# 14. Создание CSV файла с результатами
print("\nСоздание файла с результатами...")

# Создаем DataFrame с результатами
submission_df = pd.DataFrame({
    'id': test[ID_COL],
    'diagnosed_diabetes': test_predictions
})

In [ ]:
# Сохраняем в CSV файл
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
output_filename = f'submission_{timestamp}.csv'
submission_df.to_csv(output_filename, index=False)